# Test API calls

In [1]:
import os
import sys

# root path
ROOT = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Add the project root to the Python path
if ROOT not in sys.path:
    sys.path.append(ROOT)

In [2]:
import pandas as pd
import numpy as np
from src.data.api_calls import fetch_client_data, fetch_card_data

In [4]:
# fetch_client_data(1556)
fetch_card_data(1556)

Attempt 1 failed for client_id 1556 with status 429. Retrying in 2 seconds.
Attempt 2 failed for client_id 1556 with status 429. Retrying in 2 seconds.
Attempt 3 failed for client_id 1556 with status 429. Retrying in 2 seconds.


{'client_id': '1556',
 'values': {'2972': {'acct_open_date': '05/2008',
   'card_brand': 'Mastercard',
   'card_number': '5497590243197280',
   'card_on_dark_web': 'No',
   'card_type': 'Debit (Prepaid)',
   'client_id': '1556',
   'credit_limit': '$55',
   'cvv': '306',
   'expires': '07/2022',
   'has_chip': 'YES',
   'num_cards_issued': '2',
   'year_pin_last_changed': '2008'},
  '4333': {'acct_open_date': '08/2010',
   'card_brand': 'Mastercard',
   'card_number': '5225389722526534',
   'card_on_dark_web': 'No',
   'card_type': 'Debit',
   'client_id': '1556',
   'credit_limit': '$24772',
   'cvv': '891',
   'expires': '09/2023',
   'has_chip': 'YES',
   'num_cards_issued': '2',
   'year_pin_last_changed': '2010'}}}

# Load transactions data and fetch API data for unique clients (test mode)

In [67]:
transactions_df = pd.read_csv('../data/raw/transactions_data.csv', nrows=1_000_000, parse_dates=['date'])
transactions_df.drop('errors', axis=1, inplace=True)

transactions_df['amount']=transactions_df['amount'].str.strip('$').astype(np.float32)
# transactions_df['merchant_id'] = np.abs(transactions_df['merchant_id'])
transactions_df['use_chip'] = np.where(transactions_df['use_chip'] == 'Swipe Transaction', 1, 0)

transactions_df.head()
# Original shape (13305915, 12)

,id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc
0,7475327,2010-01-01 00:01:00,1556,2972,-77.00,1,59935,Beulah,ND,58523.0,5499
1,7475328,2010-01-01 00:02:00,561,4575,14.57,1,67570,Bettendorf,IA,52722.0,5311
2,7475329,2010-01-01 00:02:00,1129,102,80.00,1,27092,Vista,CA,92084.0,4829
3,7475331,2010-01-01 00:05:00,430,2860,200.00,1,27092,Crown Point,IN,46307.0,4829
4,7475332,2010-01-01 00:06:00,848,3915,46.41,1,13051,Harwood,MD,20776.0,5813


In [7]:
%%time
cat_cols = ['client_id', 'card_id', 'use_chip', 'merchant_city', 'merchant_state', 'mcc', 'merchant_id', 'zip']

for col in cat_cols:
    transactions_df[col] = transactions_df[col].astype('category')

CPU times: user 280 ms, sys: 44 ms, total: 324 ms
Wall time: 326 ms


In [8]:
%%time
cat_cols = ['client_id', 'card_id', 'use_chip', 'merchant_city', 'merchant_state', 'mcc', 'merchant_id', 'zip']

transactions_df[cat_cols] = transactions_df[cat_cols].astype('category')

CPU times: user 18.7 ms, sys: 8.03 ms, total: 26.7 ms
Wall time: 25 ms


In [9]:
transactions_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 11 columns):
 #   Column          Non-Null Count    Dtype         
---  ------          --------------    -----         
 0   id              1000000 non-null  int64         
 1   date            1000000 non-null  datetime64[ns]
 2   client_id       1000000 non-null  category      
 3   card_id         1000000 non-null  category      
 4   amount          1000000 non-null  float32       
 5   use_chip        1000000 non-null  category      
 6   merchant_id     1000000 non-null  category      
 7   merchant_city   1000000 non-null  category      
 8   merchant_state  890869 non-null   category      
 9   zip             884923 non-null   category      
 10  mcc             1000000 non-null  category      
dtypes: category(8), datetime64[ns](1), float32(1), int64(1)
memory usage: 33.7 MB


In [10]:
UNIQUE_CLIENTS = transactions_df['client_id'].unique()
# pd.Series(UNIQUE_CLIENTS).to_json('../data/processed/unique_clients.json')

In [11]:
# Temporal for test
UNIQUE_CLIENTS = UNIQUE_CLIENTS.copy()[:100]

In [12]:
UNIQUE_CLIENTS

[1556, 561, 1129, 430, 848, ..., 1209, 819, 1237, 1538, 1686]
Length: 100
Categories (1126, int64): [0, 1, 2, 3, ..., 1995, 1996, 1997, 1998]

In [ ]:
client_data_list = []
for client_id in UNIQUE_CLIENTS:
    data = fetch_client_data(client_id)
    if data:
        client_data_list.append(data)

In [17]:
card_data_list = []
for client_id in UNIQUE_CLIENTS:
    data = fetch_card_data(client_id)
    if data:
        card_data_list.append(data)

### Merge data

In [ ]:
client_df = pd.DataFrame([client['values'] for client in client_data_list])
client_df['client_id'] = [client['client_id'] for client in client_data_list]
client_df.head()

In [20]:
client_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   address            10 non-null     object
 1   birth_month        10 non-null     object
 2   birth_year         10 non-null     object
 3   credit_score       10 non-null     object
 4   current_age        10 non-null     object
 5   gender             10 non-null     object
 6   latitude           10 non-null     object
 7   longitude          10 non-null     object
 8   num_credit_cards   10 non-null     object
 9   per_capita_income  10 non-null     object
 10  retirement_age     10 non-null     object
 11  total_debt         10 non-null     object
 12  yearly_income      10 non-null     object
 13  client_id          10 non-null     object
dtypes: object(14)
memory usage: 1.2+ KB


In [19]:
cards = []
for card_entry in card_data_list:
    client_id = card_entry['client_id']
    for card_id, card_info in card_entry['values'].items():
        card_info['card_id'] = card_id
        card_info['client_id'] = client_id
        cards.append(card_info)

card_df = pd.DataFrame(cards)
card_df.head()

,acct_open_date,card_brand,card_number,card_on_dark_web,card_type,client_id,credit_limit,cvv,expires,has_chip,num_cards_issued,year_pin_last_changed,card_id
0,05/2008,Mastercard,5497590243197280,No,Debit (Prepaid),1556,$55,306,07/2022,YES,2,2008,2972
1,08/2010,Mastercard,5225389722526534,No,Debit,1556,$24772,891,09/2023,YES,2,2010,4333
2,04/2013,Mastercard,5918296616309874,No,Debit (Prepaid),561,$69,983,05/2024,YES,1,2013,2716
3,07/2006,Mastercard,5250625336912959,No,Debit,561,$26960,326,04/2022,YES,1,2010,3764
4,09/2005,Mastercard,5175842699412235,No,Credit,561,$9100,438,12/2024,YES,1,2015,4575


# After manual download of card/client data

In [5]:
from src.data.data_processing import load_transactions_data

In [57]:
client_df = pd.read_csv('../data/raw/users_data.csv')
cards_df = pd.read_csv('../data/raw/cards_data.csv', parse_dates=['expires', 'acct_open_date'])

/tmp/ipykernel_103262/3816895104.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  cards_df = pd.read_csv('../data/raw/cards_data.csv', parse_dates=['expires', 'acct_open_date'])
/tmp/ipykernel_103262/3816895104.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  cards_df = pd.read_csv('../data/raw/cards_data.csv', parse_dates=['expires', 'acct_open_date'])


In [68]:
def optimize_client_dataframe(df):
    # Define column lists
    int_cols = ['id', 'current_age', 'retirement_age', 'birth_year', 'birth_month', 'credit_score', 'num_credit_cards']
    float_cols = ['latitude', 'longitude']
    currency_cols = ['per_capita_income', 'yearly_income', 'total_debt']
    cat_cols = ['gender', 'address']
    
    # Downcast integer columns
    df[int_cols] = df[int_cols].astype('int32')
    
    # Downcast float columns
    df[float_cols] = df[float_cols].astype('float32')
    for col in float_cols:
        df[col] = round(df[col])
    
    # Strip '$' and downcast currency columns
    df[currency_cols] = df[currency_cols].str.strip('$').astype('int32')
    
    # Optimize categorical columns by converting to 'category' type
    df[cat_cols] = df[cat_cols].astype('category')
    
    return df

def optimize_card_dataframe(df):
    # Define column lists
    int_cols = ['id', 'client_id', 'num_cards_issued', 'year_pin_last_changed']
    currency_cols = 'credit_limit'
    cat_cols = ['card_brand', 'card_type', 'has_chip', 'card_on_dark_web']
    
    # Drop sensitive columns
    df = df.drop(columns=['cvv', 'card_number'])
    
    # Downcast integer columns
    df[int_cols] = df[int_cols].astype('int32')
    
    # Strip '$' and downcast currency columns
    df[currency_cols] = df[currency_cols].str.strip('$').astype('int32')
    
    # Optimize categorical columns
    df[cat_cols] = df[cat_cols].astype('category')
    
    return df

     id  client_id  card_brand        card_type    expires has_chip  \
0  4524        825        Visa            Debit 2022-12-01      YES   
1  2731        825        Visa            Debit 2020-12-01      YES   
2  3701        825        Visa            Debit 2024-02-01      YES   
3    42        825        Visa           Credit 2024-08-01       NO   
4  4659        825  Mastercard  Debit (Prepaid) 2009-03-01      YES   

   num_cards_issued  credit_limit acct_open_date  year_pin_last_changed  \
0                 2         24295     2002-09-01                   2008   
1                 2         21968     2014-04-01                   2014   
2                 2         46414     2003-07-01                   2004   
3                 1         12400     2003-01-01                   2012   
4                 1            28     2008-09-01                   2009   

  card_on_dark_web  
0               No  
1               No  
2               No  
3               No  
4               N